In [16]:
from IPython.display import display
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import gzip
import os
import csv
from alive_progress import alive_bar
from collections import defaultdict

types = defaultdict(int, token = "string")
numbers = list(range(1,535)) 
names = ['token']
names = names + numbers

directory = 'C:/Users/bincl/BA-Thesis/Dataset/2gram'
chunksize_per_df = 100000

In [17]:

#df = pd.read_csv('C:/Users/bincl/BA-Thesis/Dataset/1gram/1_20000_nopos.gz', compression= 'gzip', header=None, chunksize= 10000, names = list(range(0,535)),sep='\t',error_bad_lines=False, quoting=csv.QUOTE_NONE)
df_chunk_list = pd.read_csv('C:/Users/bincl/BA-Thesis/Dataset/3gram/3_20000_nopos_sample.zip', compression= 'zip',  header=None, index_col= [0],dtype= str,chunksize= chunksize_per_df, names = names,sep='\t', quoting=csv.QUOTE_NONE)
 

In [18]:
for chunk in df_chunk_list: 
    df_3gram_chunk = chunk 
    break 
display(chunk)

,1,2,3,4,5,6,7,8,9,10,...,525,526,527,528,529,530,531,532,533,534
token,,,,,,,,,,,,,,,,,,,,,
""" Bis seine","1736,1,1","1754,1,1","1828,1,1","1840,2,1","1841,1,1","1852,2,2","1867,1,1","1875,2,2","1876,1,1","1884,1,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
""" -- Vorgehen","1886,2,2","1902,2,2","1907,1,1","1908,1,1","1909,1,1","1916,1,1","1971,1,1","1972,1,1","1973,1,1","1975,2,2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
""" : Oesterreich","1636,1,1","1692,1,1","1743,1,1","1844,1,1","1845,6,6","1850,1,1","1854,1,1","1855,1,1","1856,2,2","1859,3,3",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
""" Geht 's","1640,1,1","1698,1,1","1824,2,2","1825,1,1","1830,2,2","1835,4,4","1836,9,7","1837,1,1","1838,1,1","1839,1,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
""" Geschlecht '","1754,1,1","1854,3,3","1858,1,1","1872,3,3","1875,1,1","1898,2,2","1899,1,1","1900,2,2","1901,2,1","1902,1,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
( Münster 1930,"1930,5,3","1931,18,15","1932,9,8","1933,6,4","1934,5,4","1935,8,5","1936,3,3","1937,6,4","1938,1,1","1939,1,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
( Besetzung durch,"1859,4,2","1878,5,5","1881,1,1","1883,1,1","1889,1,1","1907,1,1","1913,6,6","1923,1,1","1926,1,1","1930,1,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
( Hier scheint,"1751,1,1","1753,1,1","1776,1,1","1784,1,1","1789,7,7","1790,2,2","1792,2,2","1793,1,1","1797,1,1","1798,1,1",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:

def find_n_gram(direc, find):
    for filename in os.listdir(direc):
        f = os.path.join(direc, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            with gzip.open(f,'rt', encoding='utf-8') as input:
                str_arr_csv = input.readlines()
                for line in str_arr_csv:
                    token = line.split("\t")
                    if find == token[0]:
                        return line
                        
    return "line not found" 

def start_the_search(directory,search):
    result = find_n_gram(directory,search)
    
    if result == "line not found":
        print(result)
        exit()
    result = result.split("\t")
    lower_ngram = pd.Series(result[1:],dtype= "string", name = result[0])
    return lower_ngram

In [27]:
def getLowerLevelString(string):
   first, *middle, last = string.split()
   first = first + " " + ''.join(middle)
   last = ''.join(middle) + " " + last
   return first, last

def MLR(token,ngram):
    print(token)
    first,last = getLowerLevelString(token)
    print(first)
    print(last)
    first = start_the_search(directory,first)
    last = start_the_search(directory,last)
    
    X = pd.concat([first,last],axis=1)
    y = ngram.loc[token]
    reg = LinearRegression()
    reg.fit(X, y) 
    
    return reg.coef_,reg.intercept_,X

def buildApproximation(c1,c2,basevalue,df):
    df = df.astype('float')
    df['scaledFirst'] = df.iloc[:,0].apply(lambda x: x * c1) 
    df['scaledLast'] = df.iloc[:,1].apply(lambda x: x * c2) 
    df['approximation'] = df['scaledFirst'] + df['scaledLast'] + basevalue
    return df

In [21]:
def compressWithError2gram(firstN,error,df_ngram):
    df_N_gram = df_ngram.head(firstN)
    
    result = []
    all = []
    sum = []
    with alive_bar(df_N_gram.index.values.size) as bar:
        for token in df_N_gram.index.values:
            coef,intercept,dfAprox = MLR(token,df_N_gram)
            c1,c2 = coef
            df = buildApproximation(c1,c2,intercept,dfAprox)
            dfOriginal = pd.DataFrame()
            dfOriginal['values'] = df_ngram.loc[token]
            dfOriginal['values'] = dfOriginal['values'].apply(pd.to_numeric, errors='coerce')
            #dfOriginal= pd.to_numeric(dfOriginal)
            dfOriginal['zscore'] = zscore(dfOriginal)
            df['zscore'] = zscore(df['approximation'])
            
            sum.append(pd.to_numeric(dfOriginal['values']).sum()) 
            
            rmse = mean_squared_error(dfOriginal['zscore'], df['zscore'], squared = False)
            #rmse = mean_squared_error(dfOriginal, df['approximation'])
            if rmse <= error:
                result.append([token,rmse,dfOriginal['values'],dfOriginal['zscore'],df['approximation'],df['zscore']]) 
            all.append(rmse)    
            bar()
    return result, all, sum


In [28]:
result, all, sum = compressWithError2gram(chunksize_per_df,0.05, df_3gram_chunk)
print(len(result))


on 0: " Bis seine
on 0: " Bis
on 0: Bis seine
on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_aa.gz
on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_ab.gz
on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_ac.gz
on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_ad.gz
on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_ae.gz


0              1517,4,1
1              1519,1,1
2              1538,2,1
3              1541,1,1
4              1548,1,1
             ...       
404     2015,21347,9569
405     2016,19999,9056
406     2017,19710,8643
407     2018,19377,8444
408    2019,20816,8704

Name: " Bis, Length: 409, dtype: string

on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_aa.gz
on 0: C:/Users/bincl/BA-Thesis/Dataset/2gram\2_20000_nopos_ab.gz


0           1623,1,1
1           1648,1,1
2           1702,1,1
3           1719,2,2
4           1721,1,1
           ...      
223     2015,336,319
224     2016,302,294
225     2017,328,314
226     2018,291,285
227    2019,308,301

Name: Bis seine, Length: 228, dtype: string

|⚠︎                                       | (!) 0/100000 [0%] in 47.7s (0.00/s) 


TypeError: boolean value of NA is ambiguous

In [ ]:
plt.boxplot(all)
plt.show()
rmse_with_error = []

for i in result:
    rmse_with_error.append(i[1])
plt.boxplot(rmse_with_error)
plt.show()

import seaborn as sns

sns.violinplot(x= rmse_with_error, inner="point")

In [ ]:
sns.violinplot(x= all, inner_kws=dict(box_width=15, whis_width=2, color=".8"))

In [ ]:

plt.scatter(sum, all)
plt.xscale('log')
plt.show()